In [1]:
source = "Recommendations";

In [2]:
using DataFrames
import CSV
import NBInclude: @nbinclude
import Statistics: mean
@nbinclude("../InferenceAlphas/Alpha.ipynb");

In [3]:
ENV["COLUMNS"] = 999999;
ENV["LINES"] = 200;
recommend_related_series = true;

In [4]:
anime = DataFrame(CSV.File("../../data/processed_data/anime.csv", ntasks = 1))
anime_to_uid = DataFrame(CSV.File("../../data/processed_data/anime_to_uid.csv"))
anime_to_uid = innerjoin(anime_to_uid, anime, on = "anime_id");

In [5]:
rating_df = DataFrame(
    "uid" => 0:num_items()-1,
    "ranking" => read_recommendee_alpha("BPR", "all").rating,
    "explicit" => read_recommendee_alpha("Explicit", "all").rating,
    "implicit" => read_recommendee_alpha("LinearImplicit", "all").rating,
    "error_explicit" => read_recommendee_alpha("ErrorExplicit", "all").rating,
    "error_implicit" => read_recommendee_alpha("ErrorImplicit", "all").rating,
    "baseline_explicit" =>
        read_recommendee_alpha("NeuralExplicitUserItemBiases", "all").rating,
    "baseline_implicit" =>
        read_recommendee_alpha("NeuralImplicitUserItemBiases", "all").rating,
)
rating_df[:, "score"] = rating_df[:, "ranking"];

In [6]:
# evaluate our insample predictions
df = get_recommendee_split(false)
@info "RMSE: $(sqrt(mean((df.rating - rating_df.explicit[df.item]) .^ 2)))"

[ Info: 20220624 08:57:55 RMSE: 1.9887247


In [7]:
# don't recommend shows that the user has already seen before
rating_df[:, "display"] .= true
seen_items = vcat(get_recommendee_split(true).item, get_recommendee_split(false).item)
rating_df.display[seen_items] .= false

# don't recommend shows related to shows they have seen before
if !recommend_related_series
    related_series =
        get_alpha("ItemCF.Related.strict_relations", "recommendee_inference").rating .!= 0
    rating_df.display[related_series] .= false
end;

In [8]:
rec_df = innerjoin(anime_to_uid, rating_df, on = "uid");
keepcols = ["anime_id", "uid", "title", "genres", "medium"]
for x in names(rating_df)
    if x ∉ keepcols
        push!(keepcols, x)
    end
end
rec_df = rec_df[:, keepcols];

In [9]:
length(get_recommendee_split(false).item), length(get_recommendee_split(true).item)

(441, 443)

In [10]:
function pretty_display(df)
    sort(filter(x -> x.display, df), :score)
end;

In [11]:
filter(x -> x.medium == "tv", rec_df) |> pretty_display

,anime_id,uid,title,genres,medium,ranking,explicit,implicit,error_explicit,error_implicit,baseline_explicit,baseline_implicit,score,display
,Int64,Int64,String,String,String7,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Bool
1,16498,7644,Shingeki no Kyojin,"['Action', 'Drama', 'Gore', 'Military', 'Shounen', 'Survival']",tv,1.0,8.28024,10.6805,1.0673,0.259125,8.34327,18.6955,1.0,1
2,40028,5414,Shingeki no Kyojin: The Final Season,"['Action', 'Drama', 'Gore', 'Military', 'Shounen', 'Survival']",tv,2.0,9.6032,12.4646,0.585279,0.292772,8.71964,18.0355,2.0,1
3,40748,18010,Jujutsu Kaisen (TV),"['Action', 'Fantasy', 'School', 'Shounen']",tv,3.0,6.27799,11.7172,1.49129,0.287969,8.52203,18.2152,3.0,1
4,38524,12753,Shingeki no Kyojin Season 3 Part 2,"['Action', 'Drama', 'Gore', 'Military', 'Shounen', 'Survival']",tv,4.0,8.18621,11.1701,0.882901,0.292812,9.00649,18.1833,4.0,1
5,35760,19030,Shingeki no Kyojin Season 3,"['Action', 'Drama', 'Gore', 'Military', 'Shounen', 'Survival']",tv,6.0,8.59889,10.849,0.982184,0.271983,8.52402,18.2492,6.0,1
6,25777,8685,Shingeki no Kyojin Season 2,"['Action', 'Drama', 'Gore', 'Military', 'Shounen', 'Survival']",tv,8.0,8.32487,10.3852,1.08581,0.25265,8.38515,18.3866,8.0,1
7,33486,19915,Boku no Hero Academia 2nd Season,"['Action', 'School', 'Shounen', 'Super Power']",tv,10.0,7.39012,10.4206,1.26814,0.334128,8.0124,18.3299,10.0,1
8,36456,15146,Boku no Hero Academia 3rd Season,"['Action', 'School', 'Shounen', 'Super Power']",tv,11.0,7.68593,10.5959,1.17407,0.307457,7.95492,18.1933,11.0,1
9,30276,2415,One Punch Man,"['Action', 'Comedy', 'Parody', 'Seinen', 'Super Power']",tv,12.0,5.46169,9.61963,1.60591,0.319253,8.35258,18.5058,12.0,1


In [18]:
df = copy(rec_df)
df[:, "score"] = -(df[:, "explicit"] - df[:, "error_explicit"]);
filter(x -> x.explicit - x.baseline_explicit > 0.1, df) |> pretty_display

,anime_id,uid,title,genres,medium,ranking,explicit,implicit,error_explicit,error_implicit,baseline_explicit,baseline_implicit,score,display
,Int64,Int64,String,String,String7,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Bool
1,35994,10554,Satsuriku no Tenshi,"['Adventure', 'Horror', 'Psychological', 'Suspense']",tv,594.0,10.1428,5.45034,0.687939,0.337135,6.63964,16.8697,-9.45484,1
2,35120,2210,Devilman: Crybaby,"['Action', 'Avant Garde', 'Gore', 'Horror', 'Mythology', 'Supernatural']",ona,90.0,9.81699,9.73409,0.669897,0.323785,7.78099,17.4697,-9.14709,1
3,40028,5414,Shingeki no Kyojin: The Final Season,"['Action', 'Drama', 'Gore', 'Military', 'Shounen', 'Survival']",tv,2.0,9.6032,12.4646,0.585279,0.292772,8.71964,18.0355,-9.01792,1
4,37086,19190,Kakegurui××,"['Drama', 'High Stakes Game', 'Mystery', 'Psychological', 'School', 'Shounen', 'Strategy Game']",tv,195.0,9.71008,8.44181,0.78821,0.351792,7.06163,17.182,-8.92187,1
5,16592,3767,Danganronpa: Kibou no Gakuen to Zetsubou no Koukousei The Animation,"['High Stakes Game', 'Horror', 'Mystery', 'Psychological', 'School', 'Survival']",tv,218.0,9.66862,7.69672,0.760843,0.34203,7.04132,17.3171,-8.90777,1
6,37799,12517,Tokyo Ghoul:re 2nd Season,"['Action', 'Fantasy', 'Gore', 'Horror', 'Psychological', 'Seinen']",tv,345.0,9.56435,6.36644,0.668082,0.356485,6.11093,17.2498,-8.89627,1
7,38809,19744,Black'n White Jammin' Cats,['Music'],music,9841.0,9.66926,0.736837,0.787469,0.271538,6.01293,10.5983,-8.88179,1
8,4608,11604,Yuuwaku Countdown: Akira,['Hentai'],ova,9561.0,9.66816,0.178321,0.787469,0.271538,5.89745,11.0274,-8.88069,1
9,24991,12979,No Game No Life Specials,"['Comedy', 'Ecchi', 'Fantasy', 'Isekai']",special,863.0,9.3913,6.24453,0.750585,0.348045,6.4083,15.7991,-8.64071,1


In [14]:
# 	anime_id	uid	title	genres	medium	rating	alpha	std	p	nsfw
# Int64	Int64	String	String?	String7	Float32	Float32	Float32	Float32	String7
# 1	31988	11173	Hibike! Euphonium 2	['Drama', 'Music', 'School']	tv	7.38253	0.356698	0.792334	0.00168306	white
# 2	1065	967	Touch	['Drama', 'Romance', 'School', 'Shounen', 'Slice of Life', 'Sports']	tv	7.34929	0.467546	0.908755	0.00076882	white
# 3	27989	9666	Hibike! Euphonium	['Drama', 'Music', 'School']	tv	7.23239	0.571174	0.868754	0.00258483	white
# 4	1858	1694	Gakuen Utopia Manabi Straight!	['Comedy', 'School', 'Slice of Life']	tv	7.09079	1.20158	0.729596	0.00062308	white
# 5	25835	9373	Shirobako	['Comedy', 'Drama']	tv	7.14048	0.161561	0.805839	0.00261632	white
# 6	5941	4377	Cross Game	['Comedy', 'Drama', 'Romance', 'School', 'Shounen', 'Sports']	tv	7.23503	0.154975	0.9538	0.00104641	white
# 7	39570	15596	High Score Girl II	['Comedy', 'Game', 'Romance', 'School', 'Seinen']	tv	7.01997	0.324915	0.878117	0.000592566	white
# 8	1530	1388	Kanon (2006)	['Drama', 'Romance', 'Slice of Life', 'Supernatural']	tv	7.16325	0.720318	1.03834	0.00227492	white
# 9	488	458	Ichigo Mashimaro	['Comedy', 'Slice of Life']	tv	6.9334	0.724677	0.852129	0.00104676	white
# 10	122	101	Full Moon wo Sagashite	['Comedy', 'Drama', 'Music', 'Romance', 'Shoujo', 'Supernatural']	tv	6.94031	0.727181	0.869193	0.000667557	white
# 11	21877	8534	High Score Girl	['Comedy', 'Game', 'Romance', 'School', 'Seinen']	tv	6.88657	0.471743	0.941942	0.000968524	white
# 12	38993	15192	Karakai Jouzu no Takagi-san 2	['Comedy', 'Romance', 'School', 'Shounen', 'Slice of Life']	tv	6.78872	0.179552	0.993493	0.000348786	white
# 13	532	499	Bishoujo Senshi Sailor Moon S	['Drama', 'Romance', 'Shoujo']	tv	6.63147	0.41036	0.84895	0.000437212	white
# 14	135	114	Hikaru no Go	['Comedy', 'Game', 'Shounen', 'Supernatural']	tv	6.74995	0.153899	1.02637	0.000676344	white
# 15	30727	10611	Saenai Heroine no Sodatekata ♭	['Comedy', 'Ecchi', 'Harem', 'Romance', 'School']	tv	6.59874	0.364116	0.876918	0.000742145	white
# 16	593	556	Mugen no Ryvius	['Drama', 'Mecha', 'Military', 'Psychological', 'Sci-Fi', 'Space']	tv	6.68717	0.304934	0.972565	0.00101426	white
# 17	34902	12650	Tsurezure Children	['Comedy', 'Romance', 'School', 'Shounen']	tv	6.51234	0.364974	0.807823	0.00102669	white
# 18	59	40	Chobits	['Comedy', 'Drama', 'Ecchi', 'Romance', 'Sci-Fi', 'Seinen']	tv	6.6161	0.883723	0.948174	0.00179486	white
# 19	38753	15022	Araburu Kisetsu no Otome-domo yo.	['Comedy', 'Drama', 'Romance', 'School', 'Shounen']	tv	6.52573	0.495667	0.864833	0.000905996	white
# 20	31771	11089	Amanchu!	['Comedy', 'School', 'Shounen', 'Slice of Life']	tv	6.58394	0.685901	0.926502	0.000584655	white
# 21	35860	13252	Karakai Jouzu no Takagi-san	['Comedy', 'Romance', 'School', 'Shounen', 'Slice of Life']	tv	6.59876	0.432538	0.977115	0.000852553	white
# 22	427	400	Kaleido Star	['Comedy', 'Drama', 'Fantasy', 'Shoujo', 'Sports']	tv	6.57368	0.344165	0.97538	0.000534053	white
# 23	40530	16120	Jaku-Chara Tomozaki-kun	['Drama', 'Romance', 'School']	tv	6.47727	0.923203	0.894883	0.000440828	white
# 24	874	789	Digimon Tamers	['Adventure', 'Comedy', 'Drama', 'Fantasy', 'Shounen']	tv	6.52593	0.48561	0.980111	0.000362535	white
# 25	35240	12889	Princess Principal	['Action', 'Historical', 'Mystery']	tv	6.5926	0.377174	1.06195	0.000912882	white
# 26	42361	16907	Ijiranaide, Nagatoro-san	['Comedy', 'Romance', 'Slice of Life']	tv	6.60228	0.83992	1.08517	0.000529261	white
# 27	18897	7968	Nisekoi	['Comedy', 'Harem', 'Romance', 'School', 'Shounen']	tv	6.31667	0.286932	0.808886	0.00141164	white
# 28	1486	1348	Kodomo no Omocha (TV)	['Comedy', 'Drama', 'Romance', 'Shoujo']	tv	6.56859	0.231561	1.06944	0.000533452	white
# 29	35639	13136	Just Because!	['Drama', 'Romance', 'School', 'Slice of Life']	tv	6.46695	0.763056	0.99794	0.000684533	white
# 30	1222	1114	Bokura ga Ita	['Drama', 'Romance', 'Shoujo', 'Slice of Life']	tv	6.70307	1.16039	1.23905	0.000494813	white
# 31	41619	16653	Munou na Nana	['Psychological', 'Shounen', 'Super Power', 'Supernatural', 'Suspense']	tv	6.41985	0.698886	0.958501	0.000440847	white
# 32	14131	7087	Girls & Panzer	['Action', 'Military', 'School', 'Sports']	tv	6.55237	0.586552	1.13453	0.00168731	white
# 33	552	517	Digimon Adventure	['Action', 'Adventure', 'Comedy', 'Fantasy', 'Kids']	tv	6.55298	0.365117	1.13609	0.000609451	white
# 34	40685	16205	Super Cub	['School', 'Slice of Life']	tv	6.35695	0.364119	0.944397	0.000580612	white
# 35	37890	14482	Oshi ga Budoukan Ittekuretara Shinu	['Comedy', 'Girls Love', 'Music', 'Seinen', 'Slice of Life']	tv	6.20357	0.206216	0.801772	0.000379987	white
# 36	34984	12704	Koi wa Ameagari no You ni	['Drama', 'Romance', 'Seinen', 'Slice of Life']	tv	6.56196	0.419388	1.16207	0.00116559	white
# 37	3958	3399	Kannagi	['Comedy', 'School', 'Shounen', 'Supernatural']	tv	6.24875	0.494657	0.866859	0.00120494	white
# 38	31859	11120	Hai to Gensou no Grimgar	['Action', 'Adventure', 'Drama', 'Fantasy']	tv	6.24894	0.166416	0.876223	0.000963634	white
# 39	13333	6942	Tari Tari	['Music', 'School', 'Slice of Life']	tv	6.24726	0.466783	0.878362	0.00116383	white
# 40	587	550	Hanbun no Tsuki ga Noboru Sora	['Comedy', 'Drama', 'Romance']	tv	6.30107	0.38663	0.960221	0.00128816	white
# 41	8861	5539	Yosuga no Sora: In Solitude, Where We Are Least Alone.	['Drama', 'Ecchi', 'Harem', 'Romance']	tv	7.46844	3.02749	2.15606	0.00104767	gray
# 42	2986	2716	Bamboo Blade	['Comedy', 'School', 'Seinen', 'Sports']	tv	6.17986	0.452029	0.868882	0.000749638	white
# 43	35756	13204	Comic Girls	['Comedy', 'Slice of Life']	tv	6.22388	0.350058	0.919891	0.000430328	white
# 44	1087	987	Kimagure Orange☆Road	['Comedy', 'Drama', 'Romance', 'School', 'Shounen', 'Slice of Life', 'Super Power']	tv	6.38255	0.190579	1.07896	0.000837759	white
# 45	12149	6691	AKB0048	['Music', 'Sci-Fi']	tv	6.26354	0.61283	0.960399	0.00053292	white
# 46	330	306	Midori no Hibi	['Comedy', 'Drama', 'Ecchi', 'Romance', 'Shounen']	tv	6.17071	0.533026	0.875367	0.00092522	white
# 47	46093	17949	Shiroi Suna no Aquatope	['Drama', 'Slice of Life']	tv	6.36729	0.377252	1.0729	0.000547827	white
# 48	5150	3975	Hatsukoi Limited.	['Comedy', 'Romance', 'School', 'Shounen']	tv	6.09554	0.397329	0.811726	0.000535075	white
# 49	740	677	Bishoujo Senshi Sailor Moon R	['Demons', 'Romance', 'Shoujo']	tv	6.29016	0.24701	1.00655	0.000616277	white
# 50	37982	14550	Domestic na Kanojo	['Drama', 'Romance', 'School', 